In [1]:
%run generate_new_examples.ipynb     # importing functions

In [71]:
# jsonfilename = "noname.json"
limits = [0.05, 0.005, 0.0010, 0.0005, 0.0001]

# Rounding detection

This notebook's purpose is to test a following hypothesis: given a "cube" defined by real-valued coordinates from the cube algorithm, assume a roundin of these coordinates, which are in some sense *almost* a whole number. We test, whether this rounding gives us the desired coordinate of the linear combination given by the LLL. 

Note: this may not work always, as sometimes the skip is greater than 1.

In [57]:
def detect_rounding(jsonfilename):
    cases = from_json(jsonfilename)
    stats = {}
    for limit in limits:
        stats[limit] = [0, 0] 
    for case in cases:
        for i in range(len(case["lincomb_cube"])):
            
            #skip equal components
            if case["lincomb_diff"][i] == 0:
                continue
                
            for limit in limits:
                num_cube = case["lincomb_cube"][i]
                num_LLL = case["lincomb_LLL"][i]
                if ceil(num_cube, limit) or floor(num_cube, limit): # if its almost an int
                    if round(num_cube) == num_LLL:
                        stats[limit] = increment(stats, limit, 0)
                    else: 
                        stats[limit] = increment(stats, limit, 1)
    return stats

def ceil(num_cube, limit):
    # if the decimal part is almost the next integer
    # print(num_cube, 1 - abs(num_cube % 1))
    return  1 - abs(num_cube % 1) < limit 

def floor(num_cube, limit):
    # print(num_cube, abs(num_cube % 1))
    return abs(num_cube % 1) < limit

def increment(dic, limit, ind):
    if ind == 0:
        return [dic.get(limit,0)[0] + 1, dic.get(limit,0)[1]] 
    return [dic.get(limit,0)[0], dic.get(limit,0)[1] + 1]

In [73]:
for [k, [a,b]] in detect_rounding("matrices6x6-12000instances.json").items():
    print("success: {}    fail: {} for limit = {}".format(a,b, k))
    print("successful rounding ratio: ", 100 *a/(a + b).n(digits=3), " % out of all nearly integers")
    print()

success: 338    fail: 2430 for limit = 0.0500000000000000
successful rounding ratio:  12.2  % out of all nearly integers

success: 26    fail: 220 for limit = 0.00500000000000000
successful rounding ratio:  10.6  % out of all nearly integers

success: 8    fail: 44 for limit = 0.00100000000000000
successful rounding ratio:  15.4  % out of all nearly integers

success: 1    fail: 20 for limit = 0.000500000000000000
successful rounding ratio:  4.76  % out of all nearly integers

success: 0    fail: 6 for limit = 0.000100000000000000
successful rounding ratio:  0.000  % out of all nearly integers

